# MTH 652: Advanced Numerical Analysis

## Homework Assignment 1

### <span style="color:red;">Write your name here</span>

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and code.
  * The use of ChatGTP, Copilot, or other AI assistants is **not allowed**
* The code must run in Colab or JupyterHub without errors.
  * Code that does not run will not receive any credit.
  * I suggest double-checking that your code runs properly in a new session. Sometimes code can be broken but appear to work because of old state in the notebook.

### Google Colab Instructions

* After opening this assignment in Google Colab, click on **"Copy to Drive"**
* Rename the notebook to `student_name_mth_652_assignment_1.ipynb`
    * ⚠️ In the above, replace `student_name` with your name!
* Enter your name above (in the cell below "Homework Assignment")!
* When you are ready to submit your assignment, select "File -> Download -> Download .ipynb" from the Colab menu
* Upload the downloaded `.ipynb` file to Canvas

### Assignment Goals

* The purpose of this assignment is to:
    1. Implement forward Euler, backward Euler, and Crank-Nicolson time integration methods for the heat equation
    2. Study the stability and convergence behaviors of these methods

The following code (from the MTH 651 homework assignments) can be used to assemble the finite element mass and stiffness matrices.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def area(K):
    """
    Returns the area of the triangle defined by K.
    """
    M = np.array([[K[0,0], K[0,1], 1],
                  [K[1,0], K[1,1], 1],
                  [K[2,0], K[2,1], 1]])
    return 0.5 * np.linalg.det(M)

def make_stiffness(T, V, B):
    """
    Assembles the stiffness matrix on the mesh defined by (T, V). Eliminates the
    essential boundary conditions defined by B.
    """
    N = V.shape[0]
    A = np.zeros((N, N))

    for it in range(T.shape[0]):
        K = V[T[it,:],:]
        G1 = np.array([[1, 1, 1],
                       [K[0,0], K[1,0], K[2,0]],
                       [K[0,1], K[1,1], K[2,1]]])
        G2 = np.array([[0,0],[1,0],[0,1]])
        G = np.linalg.solve(G1, G2)
        A_K = area(K) * G @ G.T

        A[np.ix_(T[it,:], T[it,:])] += A_K

    A[B,:] = 0.0
    A[:,B] = 0.0
    for i in B:
        A[i,i] = 1.0

    return A

def make_mass(T, V, B):
    """
    Assembles the mass matrix on the mesh defined by (T, V). Eliminates the
    essential boundary conditions defined by B.
    """
    N = V.shape[0]
    M = np.zeros((N, N))
    for it in range(T.shape[0]):
        K = V[T[it,:],:]
        M[np.ix_(T[it,:], T[it,:])] += area(K)/12*np.array([[2,1,1],[1,2,1],[1,1,2]])

    M[B,:] = 0.0
    M[:,B] = 0.0
    for i in B:
        M[i,i] = 1.0

    return M

def square_mesh(nx):
    """
    Generates a triangular Cartesian mesh of the unit square with nx vertices in
    each dimension.

    Returns (V, T, B), where V is are the vertex coordinates, T are the triangle
    indices, and B is a list of boundary vertex indices.
    """
    x = np.linspace(0, 1, nx)
    X, Y = np.meshgrid(x, x)
    V = np.stack((X.ravel(), Y.ravel()), axis=1)

    nt = 2*(nx-1)**2
    T = np.zeros((nt, 3), int)

    for iy in range(nx - 1):
        for ix in range(nx - 1):
            v1 = ix + iy*nx
            v2 = ix + 1 + iy*nx
            v3 = ix + (iy + 1)*nx
            v4 = ix + 1 + (iy + 1)*nx
            T[2*ix + iy*2*(nx-1), :] = [v1, v2, v4]
            T[2*ix + 1 + iy*2*(nx-1), :] = [v1, v4, v3]

    B = []
    for i in range(nx):
        B.append(i)
        B.append(i + nx*(nx - 1))
    for i in range(1, nx - 1):
        B.append(nx*i)
        B.append(nx - 1 + nx*i)

    return V, T, B



def l2_error(u, u_exact, T, V):
    """
    Return the L^2 norm of the difference (u - u_exact).

    (T, V) define the triangulation of the domain. u is a vector containing
    vertex values at each of the vertices of the mesh, and u_exact is a function
    of two variables that returns the exact solution.
    """
    integral = 0.0
    for it in range(T.shape[0]):
        val = np.mean(u[T[it,:]])
        xy = np.mean(V[T[it,:],:], axis=0)
        integral += area(V[T[it,:],:]) * (val - u_exact(xy[0], xy[1]))**2
    return np.sqrt(integral)

Consider the heat equation
$$
    \begin{aligned}
        \frac{\partial u}{\partial t} - \Delta u &= f \quad\text{in $\Omega$} \\
        u(x,y,t) &= 0 \quad\text{on $\partial\Omega$} \\
        u(x,y,0) &= u_0(x,y)
    \end{aligned}
$$
Define the right-hand side $f = 0$ and initial condition
$$
    u_0(x,y) = \sin(\pi x) \sin(\pi y).
$$

#### 1. Exact Solution (1 point)

Write down the exact solution $u(x,y,t)$ to this equation.
Write a function `u_exact` that returns the exact solution.

In [2]:
def u_exact(x, y, t):
    return

#### 2. Forward Euler (3 points)

Write a function `forward_euler` that solves the heat equation on the square mesh of the domain $\Omega = [0,1]^2$ and time interval $[0,0.1]$ given parameters `nx`, `nt`.
Time integration should be performed using the forward Euler method;
the number of time steps should be `nt` (time step $\Delta t = 0.1 / n_t$).
Return the error compared with the exact solution.

If the norm of the solution at any time step is more than 10% larger than the norm of the initial conditions, return `np.nan` immediately. (This can happen if the time step does not satisfy the CFL condition, in which case the method is unstable).

In [3]:
def forward_euler(nx, nt):
    pass

#### 3. Backward Euler (3 points)

Write a function `backward_euler`, analogous to `forward_euler` from the previous question, but implementing the backward Euler method.

In [4]:
def backward_euler(nx, nt):
    pass

#### 4. Theta Method (3 points)

Write a function `theta_method` that implements the $\theta$-method, taking $\theta$ as a parameter.

In [5]:
def theta_method(theta, nx, nt):
    pass

#### 5. Convergence study (5 points)

Start with a Cartesian grid of size $3 \times 3$ (i.e. `nx = 3` in `square_mesh`) and `nt = 4`.
Run forward Euler, backward Euler and Crank-Nicolson and record the error.
Multiply `nx` and `nt` each by 2 and repeat this process; solve the problem on a total of five meshes (each refined by a factor of two from the previous mesh).

Store the errors for each of the methods in lists `e_fe`, `e_be`, and `e_cn`.
Use the provided code to output a convergence table.
Interpret the results in light of the theoretical estimates.

In [ ]:
print("FE Error      Rate      BE Error      Rate      CN Error      Rate")
print("------------------------------------------------------------------")
for i in range(len(e_fe)):
    for e in [e_fe, e_be, e_cn]:
        if i > 0 and not np.isnan(e[i]):
            rate_fmt = "{:.3f}"
            rate_l2 = np.log2(e[i-1] / e[i])
        else:
            rate_fmt = "{}"
            rate_l2 = "---  "
        print("{:9.3e}".format(e[i]), "    ", end="")
        print(rate_fmt.format(rate_l2), "    ", end="")
    print()